# 製品プライサーは継続しました

その説明から、何かがどれだけの費用がかかるかを推定できるモデル。

##データキュレーションパート2

今日は、データセットをより大きなカバレッジに拡張し、トレーニング用の優れたデータセットに作成します。

データセットはこちらです：  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023

すべての製品データセットを備えたフォルダーはこちらです。  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023/tree/main/raw/meta_categories

## liteデータセット

このノートブックは、「Day2.ipynb」に代わるものです。これは、家電製品のみの小さなデータセットを作成し、トレーニングを迅速に維持し、コストを抑えます。将来のノートブックの名前を更新して、完全なデータセットではなく「Lite」データセットを構築したことを反映する必要がある場合があります。

In [ ]:
# 輸入

import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# 環境

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# さらにいくつかのインポート -  HFログイン後にこれらを行う必要があることを指摘してくれたAlex Cに感謝します。

from items import Item
from loaders import ItemLoader

In [ ]:
%matplotlib inline

## アイテムローダーコード

Loaders.pyを見る - 私たちの生活を楽にするための便利なコードがいくつかあります

## 今、スケールアップします

大規模なホーム小売店で見つけるかもしれないすべてのアイテムのすべてのデータセットを見てみましょう - 電気、電子、オフィス、関連するが、衣服 /美しさ /本ではありません。

In [ ]:
dataset_names = [
    # 「自動車」、
    # "エレクトロニクス"、
    # 「Office_Products」、
    # 「tools_and_home_improvement」、
    # 「cell_phones_and_accessories」、
    # 「Toys_and_games」、
    "Appliances",
    # 「Musical_instruments」、
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

In [ ]:
print(f"A grand total of {len(items):,} items")

In [ ]:
# トークンカウントの分布を再度プロットします

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# 価格の分布をプロットします

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

# データセットキュレーション！

優れたデータセットを作成しました。

いくつかの最終チェックをしましょう

In [ ]:
# プロンプトの文字カウントによって価格はどのように異なりますか？

sample = items

sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# 散布図を作成します
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# ラベルとタイトルを追加します
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# プロットを表示します
plt.show()

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[50])

## 観察

Llamaトークナイザーの興味深い点は、GPT-4oで見たように、1〜999のすべての数値が1トークンにマッピングされることです。同じことは、すべての数字をトークンにマッピングするQWEN2、GEMMA、およびPHI3にも当てはまりません。これは、私たちのプロジェクトにとって少し便利であることが判明しますが、重要な要件ではありません。

＃ ついに

データをトレーニング、テスト、検証データセットに分割する時が来ました。

テスト目的でデータの5％〜10％を使用することが典型的ですが、実際にはこの時点で必要なものよりもはるかに多くあります。トレーニングには25,000ポイントを獲得し、テスト用に2,000ポイントを予約しますが、すべてを使用することはありません。


In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:25_000]
test = sample[25_000:27_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
print(train[0].prompt)

In [ ]:
print(test[0].test_prompt())

In [ ]:
# 最初の250テストポイントで価格の分布をプロットする

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# 最後に - 新しいデータセットをアップロードします

プロンプトに変換し、ハグFaceハブにアップロードします

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# リストからデータセットを作成します

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
DATASET_NAME = "ed-donner/lite-data"
dataset.push_to_hub(DATASET_NAME, private=True)

In [ ]:
# もう一つ！
# 次回、このコードをすべて実行する必要がないように、トレーニングとテストデータセットを漬けてみましょう！

with open('train_lite.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test_lite.pkl', 'wb') as file:
    pickle.dump(test, file)

## あなたのためのトドス：

 - データセットをもっと調査してください！
 - トークンザーが3桁すべての価格を1つのトークンにトークン化することを確認します